Jonathan Oleson

GMRES

MA 5631, Advanced Numerical Linear Algebra, Dr. Allan Struthers

Michigan Technological University

Adapted from a notebook by Dr. Allan Struthers

# Check Arnoldi
Modify the code here to give a dimension 30 Krylov space for a 345-dimensional 1:-2:1 Dirichlet Laplacian matrix with one extra entry A[12,17]=1.3.

In [3]:
using LinearAlgebra, KrylovKit, SparseArrays
n=345
A=spzeros(n,n);
for i in 2:n
    A[i,i]=-2.0; A[i,i-1]=1.0; A[i-1,i]=1.0
end
A[1,n]=A[n,1]=1.0;A[1,1]=-2.0
A[12,17]=1.3
v = rand(n)
v1 = copy(v)

k=30;

In [1]:
function arnoldi_jko(A, v, k)
    n = length(v)
    iterator = ArnoldiIterator(A, v)
    factorization = initialize(iterator)
    for i in 1:k-1
        expand!(iterator, factorization)
    end

    (VInt, H, res, normRes, b) = factorization

    V=zeros(n,k); Idk = Matrix(1.0I,k,k)
    v=zeros(n)
    for i in 1:k
        V[:,i]=KrylovKit.unproject!(v,VInt,Idk[:,i])
    end
    return V, H, res, normRes, b
end

arnoldi_jko (generic function with 1 method)

In [10]:
V1, H1, res1, normRes1, b1 = arnoldi_jko(A,v,k);
norm(V1'*V1 - I )
V1[:,1]./v
res1

345-element Vector{Float64}:
  0.05711134873579613
 -0.05476400891008104
  0.128984650969321
  0.030280858436243428
 -0.02671418676815766
  0.11732172958420413
 -0.038050566209011934
  0.02726686209472676
  0.000541027651618252
 -0.06997367676763323
  0.007616977789766557
  0.09256723930501362
  0.028745492860796175
  ⋮
  0.004260798490858781
  0.07484667399015357
  0.024413750666810535
  0.09209760198192528
  0.06629316913121143
 -0.1397027605067228
 -0.015839169129989866
  0.003826255120290359
  0.10511474058623724
  0.005922732570972464
  0.053937608534075435
 -0.0197967254291019

In [58]:
# test stuff outside of a function
iterator = ArnoldiIterator(A, v)
factorization = initialize(iterator)
for i in 1:k-1
    expand!(iterator, factorization)
end

(VInt, H, res, normRes, b) = factorization

V=zeros(n,k); Idk = Matrix(1.0I,k,k)
v=zeros(n)
for i in 1:k
    V[:,i]=KrylovKit.unproject!(v,VInt,Idk[:,i])
end

In [59]:
[norm(V-V1) norm(H-H1) norm(res-res1) norm(normRes-normRes1) norm(b-b1)]

1×5 Matrix{Float64}:
 0.0  0.0  0.0  0.0  0.0

# Build a GMRES Function

In [60]:
function gmres_jko(A, v, k)
    V, H, res, normRes, b = arnoldi_jko(A,v,k)
    
    H_hat = zeros(k+1,k)
    H_hat[1:k,:] = Matrix(H)
    H_hat[k+1,:] = normRes*b'
    
    Usvd,Ssvd,Vsvd = svd(H_hat)
    e1 = zeros(k+1,1)
    e1[k+1] = 1
    ym = Vsvd*inv(Diagonal(Ssvd))*Usvd'*(normRes*e1)
    xm = V*ym
    
    return xm
end

gmres_jko (generic function with 1 method)

# Solve A.x = b

Build $H_{hat}$.

In [25]:
V1, H1, res1, normRes1, b1 = arnoldi_jko(A,v,k);

H_hat = zeros(k+1,k)
H_hat[1:k,:] = Matrix(H1)
H_hat[k+1,:] = normRes1*b1';
H_hat;
A*V1 - [V1 res1]*H_hat

345×30 Matrix{Float64}:
  6.93889e-18   2.77556e-17  -2.77556e-17  …   1.38778e-16  -0.00259338
  0.0           0.0           6.93889e-18     -5.55112e-17   0.00248679
  5.20417e-18   6.93889e-18   1.73472e-18     -3.46945e-18  -0.00585708
  0.0          -1.38778e-17   0.0              1.04083e-17  -0.00137503
  6.93889e-18   0.0           0.0              2.77556e-17   0.00121307
 -2.71051e-19   1.38778e-17   0.0          …  -2.77556e-17  -0.00532748
 -6.93889e-18   0.0           1.38778e-17      1.73472e-18   0.00172784
  0.0           0.0          -1.73472e-18      1.38778e-17  -0.00123817
  1.04083e-17   1.38778e-17  -1.56125e-17     -1.38778e-17  -2.45676e-5
 -3.46945e-18   6.93889e-18   0.0              2.77556e-17   0.00317745
  1.38778e-17   0.0           2.77556e-17  …   3.46945e-18  -0.00034588
  2.77556e-17   2.77556e-17   5.55112e-17     -2.77556e-17  -0.0042034
  0.0           0.0           0.0             -4.16334e-17  -0.00130531
  ⋮                                      

Recall that $x_m = x_0 + V_m.y_m$ and $y_m$ minimizes the norm of $\beta*e_1-H_{hat}.y$. Find $y_m$ with an SVD least-squares approach, then find $x_m$.

In [62]:
Usvd,Ssvd,Vsvd = svd(H_hat)
e1 = zeros(k+1,1)
e1[k+1] = 1
ym = Vsvd*inv(Diagonal(Ssvd))*Usvd'*(normRes*e1)
xm = V*ym;

In [66]:
xm1 = gmres_jko(A, v1, k)
norm(xm-xm1)

0.0

In [65]:
b_vec = ones(n,1)
norm(b_vec-A*x)

18.577483752867582

# Test stuff
Check whether the GMRES code will work on a matrix with clustered eigenvalues.

In [32]:
n = 20
Lambda_B = spzeros(n,n)
for i in 1:n
    if i<6; Lambda_B[i,i] = 1;
    elseif i<13; Lambda_B[i,i] = 3;
    else Lambda_B[i,i] = 5;
    end
end
Q = qr(rand(n,n)).Q
B = Q*Lambda_B*Q'
b_vec = ones(n,1)
w = vec(b_vec/norm(b_vec))
j = 20
# x = gmres_jko(B, w, j)
# norm(b_vec-B*x)
# eigen(B).values
Q

20×20 LinearAlgebra.QRCompactWYQ{Float64, Matrix{Float64}}:
 -0.15066     -0.089809    0.187998     …  -0.0392888   -0.354406
 -0.137322    -0.18048     0.201906         0.0728892   -0.104311
 -0.0523196   -0.349954   -0.179753        -0.0481637    0.0605832
 -0.376285     0.0916722  -0.0122019        0.0137714   -0.0971442
 -0.00701816  -0.634664    0.0481549        0.223155     0.17019
 -0.221528     0.0437941  -0.066078     …   0.0580552    0.0334094
 -0.0365726   -0.0775113  -0.299787        -0.279808     0.117791
 -0.30917      0.160663   -0.219528        -0.239825    -0.201682
 -0.25152      0.126208   -0.283309         0.365214    -0.131477
 -0.339325    -0.0126484   0.473346        -0.00339754   0.115123
 -0.334746     0.214965   -0.235583     …   0.0995228    0.317293
 -0.194898     0.262056    0.134856         0.285932     0.391824
 -0.190705     0.0464493  -0.0640216       -0.531061     0.0609135
 -0.287012    -0.198034   -0.088887        -0.237058     0.318817
 -0.0368911  

In [116]:
eigen(B).values

20-element Vector{Float64}:
 0.9999999999999981
 0.9999999999999996
 1.0
 1.0
 1.000000000000001
 2.9999999999999996
 3.0
 3.0000000000000004
 3.0000000000000013
 3.0000000000000013
 3.0000000000000018
 3.0000000000000027
 4.999999999999996
 4.9999999999999964
 4.999999999999998
 5.000000000000002
 5.000000000000003
 5.0000000000000036
 5.0000000000000036
 5.000000000000004